# Self study 2


In this self-study we build an index that supports Boolean search over the web pages that you crawl with the crawler from the 1st self study. You can continue to just extract the titles of the web-pages you crawl, or you can be more adventurous and look at the whole text that you get from the .get_text() method of a BeautifulSoup parser. In either case, the collection of texts from the crawled web-pages is you corpus. You should then:

- construct the vocabulary of terms for your corpus
- build an 'inverted' index for your vocabulary
- implement Boolean search for your index (perhaps only for a limited set of Boolean queries)

In [21]:
# Some things already used in self study 1:
import requests
from bs4 import BeautifulSoup
from time import sleep
from urllib.robotparser import RobotFileParser
from urllib.parse import urlparse
from datetime import datetime, timedelta
from itertools import count
import pandas as pd
import numpy as np
import random
import nltk
nltk.download('punkt')
from nltk.stem.snowball import SnowballStemmer
import codecs


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\johan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


What is most useful for you depends on which websites you crawl. It is not essential for the exercise that the stemming always is the best possible ...!

In [22]:
index_frame = pd.read_csv('index.csv')
index_arr = index_frame.to_dict('records')

index_arr = index_arr[0]
index_arr.pop('Unnamed: 0')

for key in index_arr.keys():
    print(f"{key}: {index_arr[key]}")

https://www.tv2.dk: TV 2 - bedst pÃ¥ breaking og live
https://www.bt.dk: B.T. Nyheder | Læs nyhederne på bt.dk
https://www.berlingskemedia.dk/handelsbetingelser: Generelle handelsbetingelser
https://www.bt.dk/cookiedeklaration: Cookiedeklaration
https://www.dr.dk: DR | Nyheder - Breaking - TV - Radio
https://www.bt.dk/kongehusetbagkulissen: Kongehuset bag kulissen
https://www.weekendavisen.dk/mine-sider/kundeservice: Mine sider | Weekendavisen
https://www.weekendavisen.dk/: Weekendavisen | Weekendavisen
https://www.berlingske.dk/mine-sider/kundeservice: Nyheder og seneste nyt fra Berlingske - Berlingske.dk
https://www.weekendavisen.dk/titusinde: Podcast: Lone Frank & de titusinde sjæle | Weekendavisen
https://www.weekendavisen.dk/en-ny-ladning-roeverhistorier: En ny ladning Røverhistorier | Weekendavisen
https://www.bt.dk/mine-sider/kundeservice: BT | Mine sider
https://www.dr.dk/om-dr/vilkaar-paa-drdk: Vilkår på dr.dk | Om DR | DR
https://www.dr.dk/om-dr/job: Job | DR
https://www.euro

In [23]:
for key in index_arr.keys():
    print(f"{key}: {index_arr[key]}")

https://www.tv2.dk: TV 2 - bedst pÃ¥ breaking og live
https://www.bt.dk: B.T. Nyheder | Læs nyhederne på bt.dk
https://www.berlingskemedia.dk/handelsbetingelser: Generelle handelsbetingelser
https://www.bt.dk/cookiedeklaration: Cookiedeklaration
https://www.dr.dk: DR | Nyheder - Breaking - TV - Radio
https://www.bt.dk/kongehusetbagkulissen: Kongehuset bag kulissen
https://www.weekendavisen.dk/mine-sider/kundeservice: Mine sider | Weekendavisen
https://www.weekendavisen.dk/: Weekendavisen | Weekendavisen
https://www.berlingske.dk/mine-sider/kundeservice: Nyheder og seneste nyt fra Berlingske - Berlingske.dk
https://www.weekendavisen.dk/titusinde: Podcast: Lone Frank & de titusinde sjæle | Weekendavisen
https://www.weekendavisen.dk/en-ny-ladning-roeverhistorier: En ny ladning Røverhistorier | Weekendavisen
https://www.bt.dk/mine-sider/kundeservice: BT | Mine sider
https://www.dr.dk/om-dr/vilkaar-paa-drdk: Vilkår på dr.dk | Om DR | DR
https://www.dr.dk/om-dr/job: Job | DR
https://www.euro

In [24]:
dstemmer=SnowballStemmer("danish")
a_file = codecs.open("stopord.txt", "r", "utf-8")
file_contents = a_file.read()
contents_split = file_contents.splitlines()
stop_words = []
for word in contents_split:
    stop_words.append(dstemmer.stem(word))
extra_words = ["|", ",", ".", "-", "!"]
for word in extra_words:
    stop_words.append(word)

for word in stop_words:
    print(word)
a_file.close()

ad
af
aldr
alen
all
all
alligevel
alt
altid
and
and
andr
at
bag
bar
beg
bl.a.
bland
blev
bliv
bliv
burd
bør
ca.
da
de
dem
den
den
den
der
dereft
der
derfor
derfra
deri
dermed
derpå
derved
det
det
dig
din
din
dis
dit
dog
du
eft
egen
ej
ell
ell
en
end
endnu
ene
enest
enhv
ens
ent
er
et
f.eks.
far
fem
fik
fir
fler
flest
flest
for
foran
fordi
for
fra
fx
få
får
før
først
gennem
gjord
gjort
god
godt
gør
gør
gør
ham
han
han
har
havd
hav
hej
hel
hel
helt
hen
hend
hend
henov
her
hereft
heri
hermed
herpå
hos
hun
hvad
hvem
hver
hvilk
hvilk
hvilk
hvis
hvor
hvordan
hvoreft
hvorfor
hvorfra
hvorh
hvori
hvorimod
hvornår
hvorved
i
igen
igennem
ikk
imellem
imen
imod
ind
indtil
ing
int
ja
jeg
jer
jer
jo
kan
kom
kom
kom
kun
kun
lad
lang
lav
lav
lav
lidt
lig
ligesom
lil
læng
man
mand
mang
med
meg
mellem
men
men
mer
mest
mig
min
mindr
mindst
min
mit
mod
må
måsk
ned
nej
nem
ni
nog
nogensind
nog
nogl
nok
nu
ny
nyt
når
nær
næst
næst
og
også
okay
om
omkring
op
os
ott
over
overalt
pga.
på
sam
sam
se
sek
selv
sel

In [29]:
stemmed_index = {}
for key in index_arr.keys():
    try:
        tokens = nltk.word_tokenize(index_arr[key])
        local_tokens = []
        for token in tokens:
            if token not in stop_words:
                local_tokens.append(token)
        stemmed_index[key] = local_tokens
    except:
        print(f'error with:  {key}')
        continue

#used to give numbers
numbered_index = {}
i = 1
for url in stemmed_index.keys():
    stemmed_index[url] = {'tokens' : stemmed_index[url], 'id' : i}
    numbered_index[i] = url
    i += 1
for key in numbered_index.keys():
    print(f'{key}: {numbered_index[key]}')

error with:  https://www.nyheder.aau.dk/2020
error with:  https://www.sundhed.dk/borger/guides/sundhedstilbud/?SearchTerm=&searchType=searchTerm&Page=1&PageSize=10&Tilbudstype=-1&Region=-1&Udbyder=-1
error with:  https://www.aub.aau.dk/find-materiale/databaser/info?id=1352281
error with:  https://www.nordiskemediermedlemskab.dk
1: https://www.tv2.dk
2: https://www.bt.dk
3: https://www.berlingskemedia.dk/handelsbetingelser
4: https://www.bt.dk/cookiedeklaration
5: https://www.dr.dk
6: https://www.bt.dk/kongehusetbagkulissen
7: https://www.weekendavisen.dk/mine-sider/kundeservice
8: https://www.weekendavisen.dk/
9: https://www.berlingske.dk/mine-sider/kundeservice
10: https://www.weekendavisen.dk/titusinde
11: https://www.weekendavisen.dk/en-ny-ladning-roeverhistorier
12: https://www.bt.dk/mine-sider/kundeservice
13: https://www.dr.dk/om-dr/vilkaar-paa-drdk
14: https://www.dr.dk/om-dr/job
15: https://www.euroinvestor.dk/
16: https://www.pressenaevnet.dk/
17: https://www.dr.dk/nyheder/nyh

In [30]:
for key in stemmed_index.keys():
    print(f"{key}: {stemmed_index[key]['tokens']} [{stemmed_index[key]['id']}]")

https://www.tv2.dk: ['TV', '2', 'bedst', 'pÃ¥', 'breaking', 'live'] [1]
https://www.bt.dk: ['B.T', 'Nyheder', 'Læs', 'nyhederne', 'bt.dk'] [2]
https://www.berlingskemedia.dk/handelsbetingelser: ['Generelle', 'handelsbetingelser'] [3]
https://www.bt.dk/cookiedeklaration: ['Cookiedeklaration'] [4]
https://www.dr.dk: ['DR', 'Nyheder', 'Breaking', 'TV', 'Radio'] [5]
https://www.bt.dk/kongehusetbagkulissen: ['Kongehuset', 'kulissen'] [6]
https://www.weekendavisen.dk/mine-sider/kundeservice: ['Mine', 'sider', 'Weekendavisen'] [7]
https://www.weekendavisen.dk/: ['Weekendavisen', 'Weekendavisen'] [8]
https://www.berlingske.dk/mine-sider/kundeservice: ['Nyheder', 'seneste', 'Berlingske', 'Berlingske.dk'] [9]
https://www.weekendavisen.dk/titusinde: ['Podcast', ':', 'Lone', 'Frank', '&', 'titusinde', 'sjæle', 'Weekendavisen'] [10]
https://www.weekendavisen.dk/en-ny-ladning-roeverhistorier: ['En', 'ladning', 'Røverhistorier', 'Weekendavisen'] [11]
https://www.bt.dk/mine-sider/kundeservice: ['BT', 

In [31]:
simple_inverted_index = {}

for url in stemmed_index.keys():
    for token in stemmed_index[url]['tokens']:
        if token in simple_inverted_index.keys():
            if stemmed_index[url]['id'] not in simple_inverted_index[token]:
                simple_inverted_index[token].append(stemmed_index[url]['id'])
        else:
            simple_inverted_index[token] = [stemmed_index[url]['id']]

for key in simple_inverted_index.keys():
    print(f"{key}: {simple_inverted_index[key]}")
    

TV: [1, 5, 861]
2: [1]
bedst: [1]
pÃ¥: [1]
breaking: [1]
live: [1]
B.T: [2]
Nyheder: [2, 5, 9, 81, 224, 543, 802, 804, 808, 885]
Læs: [2, 148, 538]
nyhederne: [2]
bt.dk: [2]
Generelle: [3]
handelsbetingelser: [3]
Cookiedeklaration: [4, 93]
DR: [5, 13, 14, 17, 20, 27, 81, 242, 246]
Breaking: [5]
Radio: [5]
Kongehuset: [6, 459]
kulissen: [6]
Mine: [7, 12]
sider: [7, 12, 742]
Weekendavisen: [7, 8, 10, 11, 18, 22, 25, 26, 56, 68, 145, 188, 361, 389, 396, 554, 641, 712, 857, 894]
seneste: [9, 224]
Berlingske: [9, 148, 224]
Berlingske.dk: [9, 224]
Podcast: [10, 21, 65, 78]
:: [10, 16, 20, 27, 47, 51, 194, 204, 237, 278, 286, 342, 458, 479, 480, 515, 518, 681, 698, 715, 794, 833, 836, 849, 851, 852, 901]
Lone: [10]
Frank: [10]
&: [10, 251, 289, 574, 588, 622, 679, 684, 691, 751, 815, 899, 900, 907, 921]
titusinde: [10]
sjæle: [10]
En: [11, 68, 836]
ladning: [11]
Røverhistorier: [11, 22]
BT: [12, 47, 51, 194, 204, 237, 247, 464, 479, 480, 515, 518, 550, 681, 833]
Vilkår: [13]
dr.dk: [13]
Om: [

In [32]:
#Single word search

def convert_postings_to_urls(found_postings):
    found_urls = []
    for posting in found_postings:
        found_urls.append(numbered_index[posting])
    return found_urls

def single_word_search(search_word, want_postings = False):
    found_postings = []
    for token in simple_inverted_index.keys():
        if (token == search_word):
            found_postings = simple_inverted_index[token]
    if want_postings:
        return found_postings
    if found_postings:
        return convert_postings_to_urls(found_postings)
    return []
    

for url in single_word_search('DR'):
    print(url)

https://www.dr.dk
https://www.dr.dk/om-dr/vilkaar-paa-drdk
https://www.dr.dk/om-dr/job
https://www.dr.dk/nyheder/nyhedsbreve/tilmelding
https://www.dr.dk/nyheder/nyhedsbreve/seneste/dr-deler
https://www.dr.dk/nyheder/nyhedsbreve/seneste/om-dr
https://www.dr.dk/etik-og-rettelser/brugernes-redaktoer/nyheder/roer-blot-ikke-ved-skihop-fra-garmish-partenkirchen
https://www.dr.dk/etik-og-rettelser/brugernes-redaktoer
https://www.dr.dk/om-dr/organisationen/publikationer


In [41]:
#Search

def OR_merge(found_urls_one, found_urls_two):
    return list(set(found_urls_one + found_urls_two))

def AND_merge(found_urls_one, found_urls_two):
    return list(set(found_urls_one).intersection(set(found_urls_two)))

def AND_NOT_merge(found_urls_one, found_urls_two):
    a = set(found_urls_one)
    b = set(found_urls_two)    
    return list(a-b)

def Search(word_one, search_type, word_two):
    found_urls_one = single_word_search(word_one, want_postings=True)
    found_urls_two = single_word_search(word_two, want_postings=True)
    if(search_type == 'AND'):
        return convert_postings_to_urls(AND_merge(found_urls_one, found_urls_two))
    elif(search_type == 'OR'):
        return convert_postings_to_urls(OR_merge(found_urls_one, found_urls_two))
    elif(search_type == 'AND NOT'):
        return convert_postings_to_urls(AND_NOT_merge(found_urls_one, found_urls_two))
    else:
        return ['Invalid search type']

for url in Search('DR', 'AND NOT', 'installation'):
    print(url)